In [1]:
import ast
import pandas as pd
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

### Useful functions

In [2]:
def get_df_from_csv(path_file: str) -> pd.DataFrame:
    return pd.read_csv(path_file)

In [3]:
def display_df(df: pd.DataFrame) -> None:
    i = 0
    iteration = 2
    while i < len(df.columns):
        print(df.iloc[:, i:i+iteration].head())
        i += iteration

In [4]:
def get_translation_dict(df: pd.DataFrame, name_col: str, id_name_col: str) -> dict:
    translation_dict = dict()
    for row_collections in df[name_col]:
        if isinstance(row_collections, str):
            # if float, row_collections = Nan
            for row_collection in ast.literal_eval(row_collections):
                id_collection = row_collection[id_name_col]
                name_collection = row_collection['name']
                translation_dict[id_collection] = name_collection
    return translation_dict

## EXTRACTING

In [5]:
path_file = r'data\train.csv'
train_df = get_df_from_csv(path_file)

## EXPLORATY DATA ANALYSIS

In [6]:
display_df(train_df)

   id                              belongs_to_collection
0   1  [{'id': 313576, 'name': 'Hot Tub Time Machine ...
1   2  [{'id': 107674, 'name': 'The Princess Diaries ...
2   3                                                NaN
3   4                                                NaN
4   5                                                NaN
     budget                                             genres
0  14000000                     [{'id': 35, 'name': 'Comedy'}]
1  40000000  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
2   3300000                      [{'id': 18, 'name': 'Drama'}]
3   1200000  [{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...
4         0  [{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...
                            homepage    imdb_id
0                                NaN  tt2637294
1                                NaN  tt0368933
2  http://sonyclassics.com/whiplash/  tt2582802
3         http://kahaanithefilm.com/  tt1821480
4                                NaN  tt

In [7]:
train_df.shape
# 3000 rows and 23 columns
train_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
id                       3000 non-null int64
belongs_to_collection    604 non-null object
budget                   3000 non-null int64
genres                   2993 non-null object
homepage                 946 non-null object
imdb_id                  3000 non-null object
original_language        3000 non-null object
original_title           3000 non-null object
overview                 2992 non-null object
popularity               3000 non-null float64
poster_path              2999 non-null object
production_companies     2844 non-null object
production_countries     2945 non-null object
release_date             3000 non-null object
runtime                  2998 non-null float64
spoken_languages         2980 non-null object
status                   3000 non-null object
tagline                  2403 non-null object
title                    3000 non-null object
Keywords             

In [8]:
print(f"Unique ids : {train_df['id'].value_counts().size}")

Unique ids : 3000


In [9]:
for column in train_df.columns:
    series_unique_values = train_df[column].value_counts()
    nb_unique_values = series_unique_values.size
    print(f'"{column}"')
    if nb_unique_values == train_df.shape[0]:
        print(f'Only unique values, example of value: {train_df.loc[0, column]}\n')
    else:
        print(f'Nb of unique values: {nb_unique_values}')
        print(f'{train_df[column].value_counts()} \n')

"id"
Only unique values, example of value: 1

"belongs_to_collection"
Nb of unique values: 422
[{'id': 645, 'name': 'James Bond Collection', 'poster_path': '/HORpg5CSkmeQlAolx3bKMrKgfi.jpg', 'backdrop_path': '/6VcVl48kNKvdXOZfJPdarlUGOsk.jpg'}]                          16
[{'id': 9735, 'name': 'Friday the 13th Collection', 'poster_path': '/uobgqpLQff9WvxGKE2OSvXv1RHm.jpg', 'backdrop_path': '/c7pMKwv5NzIN6N3KM4L8fYMTtPw.jpg'}]                    7
[{'id': 937, 'name': 'The Pink Panther (Original) Collection', 'poster_path': '/xu6gZ0zGUSR8SC2QLKFaUrX2niM.jpg', 'backdrop_path': '/6KE7AguGR3UqwUn5RkzkoBc3MLy.jpg'}]         6
[{'id': 34055, 'name': 'Pokémon Collection', 'poster_path': '/j5te0YNZAMXDBnsqTUDKIBEt8iu.jpg', 'backdrop_path': '/iGoYKA0TFfgSoZpG2u5viTJMGfK.jpg'}]                           5
[{'id': 9338, 'name': 'Police Academy Collection', 'poster_path': '/1VRdXVy2FYlX4EtllnLKWK6kYDJ.jpg', 'backdrop_path': '/RhpI828r9EKPV4x0pyHiqCDZDQ.jpg'}]                      5
[{'id': 8354, '

Everyone deserves a chance to follow their dreams, but some people only get one shot. Tyler Gage is a rebel from the wrong side of Baltimore's tracks and the only thing that stands between him and an unfulfilled life are his dreams of one day making it out of there. Nora is a privileged ballet dancer attending Baltimore's ultra-elite Maryland School of the Arts                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       1
Based on the real-life Richard Speck murders, amoral, nearly psychotic killer Warren Stacey (Gene Davis) is a serial killer who has murdered a number of women; he s


"revenue"
Nb of unique values: 2850
6000000       10
11000000      10
10000000       9
500000         8
12000000       6
5000000        6
13000000       5
1000000        5
14000000       5
4000000        5
20000000       5
1              4
1500000        4
25000000       4
2500000        4
17000000       4
102000000      4
25             3
1300000        3
18000000       3
2200000        3
4100000        3
7000000        3
7900000        3
8              3
8500000        3
7800000        3
3000000        3
10400000       3
3              3
              ..
33000337       1
641            1
35456          1
2683519        1
254134910      1
6488144        1
3662459        1
108539911      1
1506249360     1
50000000       1
9448082        1
41480851       1
40590000       1
288752301      1
112462508      1
3665069        1
10609321       1
56359980       1
4635300        1
53898845       1
2042530        1
28900000       1
19776159       1
8303261        1
1548955        1
33697647   

#### Ideas of influence on the overall worldwide box office revenue:
* belong to a famous collection (col: belongs_to_collection)
* budget
* genres (col: genres, which can be one-hot-encoding)
* original_language
* original_title
* production_companies (col: production_companies, which can be one-hot-encoding)
* production_countries (col: production_countries, which can be one-hot-encoding)
* release_date --> to transform into dates, to get the year, the month and the week day of release
* runtime
* spoken_languages (col: spoken_languages, which can be one-hot-encoding)
* status: Released/Rumored --> check that Rumored, there is 0 revenue.
* keywords description (col: Keywords, which can be one-hot-encoding) --> but sometimes, can be too specific.

#### Column not very useful / to investigate to understand ?
* col: homepage --> what for old movies
* col: imdb_id
* col: overview --> need to analyse a whole summary --> sentiment analyis ? key_words ?
* col: tagline --> need to analyse a whole sentence --> sentiment analyis ? key_words ?
* col: popularity --> what scale ? 
* col: poster_path --> maybe if not poster_path, not a lot of advertisement
* col: title --> it is useful or do we need to transform it --> need to analyse a whole sentence --> key_words ?
* col: cast --> can be interpreted to detect a famous actor (therefore, compared with previous movies, so sort movies by date)

#### Output col: revenue

#### Analysis of col 'status'

In [10]:
column = 'status'
print(train_df[column].value_counts())
train_df.loc[(train_df[column] == 'Rumored'), 'revenue']

Released    2996
Rumored        4
Name: status, dtype: int64


609       273644
1007          60
1216    13418091
1618      229000
Name: revenue, dtype: int64

#### Analysis of one-hot-encoding columns

##### belongs_to_collection col

In [11]:
column = 'belongs_to_collection'
translation_collections_dict = get_translation_dict(train_df, column, 'id')
print(len(translation_collections_dict)) #422 collections

422


##### genres

In [12]:
column = 'genres'
translation_genres_dict = get_translation_dict(train_df, column, 'id')
print(len(translation_genres_dict)) #20 genres

20


##### production_companies

In [13]:
column = 'production_companies'
translation_prod_companies_dict = get_translation_dict(train_df, column, 'id')
print(len(translation_prod_companies_dict)) #3712 production companies

3712


##### production_countries

In [14]:
column = 'production_countries'
translation_prod_countries_dict = get_translation_dict(train_df, column, 'iso_3166_1')
print(len(translation_prod_countries_dict)) #74 production countries

74


##### spoken_languages

In [15]:
column = 'spoken_languages'
translation_spoken_lang_dict = get_translation_dict(train_df, column, 'iso_639_1')
print(len(translation_spoken_lang_dict)) #79 spoken languages

79


##### keywords

In [16]:
column = 'Keywords'
translation_keywords_dict = get_translation_dict(train_df, column, 'id')
print(len(translation_keywords_dict)) #7400 keywords --> too many keywords for a first application

7400


##### original_language

In [17]:
column = 'original_language'
list_languages = list(train_df[column].value_counts().index)
len(list_languages) # 36 languages

36

#### release_date col

In [18]:
# convert it to datetime
train_df['release_date'] = pd.to_datetime(train_df['release_date'])
# add columns year and month
train_df['year'] = train_df['release_date'].map(lambda date: date.year)
train_df['month'] = train_df['release_date'].map(lambda date: date.month)
train_df['dayofweek'] = train_df['release_date'].map(lambda date: date.dayofweek)

In [19]:
print(len(train_df.columns))
print(train_df.columns[0:10])

26
Index(['id', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity'],
      dtype='object')


In [20]:
columns = ['id', 'belongs_to_collection', 'budget', 'genres', 'original_language', 'popularity', 'production_companies', 
           'production_countries', 'release_date', 'runtime', 'spoken_languages', 
           'Keywords', 'revenue']

#### Production companies

In [21]:
set_unique_combination_values = train_df['production_companies'].value_counts()
print(f'For 3000rows, nb of production companies: {len(set_unique_combination_values)}')

For 3000rows, nb of production companies: 2383


In [22]:
dict_production_companies = get_translation_dict(train_df, 'production_companies', 'id')
# 3695 production_companies
# 2383 combinations of production_companies (for 3000 movies)

In [23]:
# test for Paramount Production company
test_paramount_appearance = {name for name in dict_production_companies.values() if 'Paramount' in name}
test_paramount_appearance

{'Paramount',
 'Paramount Animation',
 'Paramount Classics',
 'Paramount Pictures',
 'Paramount Vantage'}

In [24]:
# try to simplify complicated names
useless_info_inside_title = ['Picture', 'Image', 'Animation', 'Classic', 'Vantage', 'Film', 'Production', 'Entertainment', 
                             'Studio', 'Inc.', 'Inc', ', The', 'L.P.', 'Company']

def simplify_prod_companies_names(list_unique_prod_companies: [str], list_useless_info: [str]) -> dict:
    dict_simplified_name_prod_companies = dict()
    for complicated_name in list_unique_prod_companies:
        simplified_name = complicated_name
        for replacement in [(info + 's', '') for info in list_useless_info]:
            simplified_name = simplified_name.replace(*replacement)
        for replacement in [(info, '') for info in list_useless_info]:
            simplified_name = simplified_name.replace(*replacement)
        simplified_name = simplified_name.strip()
        dict_simplified_name_prod_companies[complicated_name] = simplified_name
    return dict_simplified_name_prod_companies

In [25]:
list_unique_prod_companies = list(dict_production_companies.values())
dict_simplified_name_prod_companies = simplify_prod_companies_names(list_unique_prod_companies, useless_info_inside_title)

In [26]:
# plot the frequency of appearances of production companies
# get simpified names and frequencies
simplified_production_name_freq = dict()
for collection, frequency in set_unique_combination_values.iteritems():
    for production in ast.literal_eval(collection):
        complicated_production_name = production['name']
        previous_frequency = simplified_production_name_freq.get(dict_simplified_name_prod_companies[complicated_production_name], 0)
        simplified_production_name_freq[dict_simplified_name_prod_companies[complicated_production_name]] = previous_frequency + frequency
        
iter_nb_companies = 0
for v in simplified_production_name_freq.values():
    if v > 15:
        iter_nb_companies += 1
print(f'For 3000rows, nb of production companies whose frequency is bigger than 15 times :{iter_nb_companies}')

famous_prod_companies =[name for name, frequency in simplified_production_name_freq.items() if frequency > 15]
print(famous_prod_companies)

For 3000rows, nb of production companies whose frequency is bigger than 15 times :47
['Paramount', 'Universal', 'Metro-Goldwyn-Mayer (MGM)', 'Warner Bros.', 'Twentieth Century Fox  Corporation', 'Columbia', 'New Line Cinema', 'Touchstone', 'TriStar', 'Orion', 'Walt Disney', 'United Artists', 'Columbia  Corporation', 'Miramax', 'Morgan Creek', 'Hollywood', 'Focus Features', 'Dimension', 'Amblin', 'Fox 2000', 'Imagine', 'Summit', 'Blumhouse', 'Working Title', 'DreamWorks SKG', 'Regency Enterprises', 'Epsilon Motion', 'New Regency', 'BBC', 'Castle Rock', 'Canal', 'Lionsgate', '4', 'Canal+', 'Fox Searchlight', 'Village Roadshow', 'Original', 'Dune', 'Millennium', 'TF1', 'Spyglass', 'TSG', 'Participant Media', 'Screen Gems', 'Legendary', 'Relativity Media', 'Babelsberg']


#### Keywords

In [27]:
set_unique_combination_values = train_df['Keywords'].value_counts()
print(f'For 3000rows, nb of production companies: {len(set_unique_combination_values)}')

For 3000rows, nb of production companies: 2648


In [28]:
dict_keywords = get_translation_dict(train_df, 'Keywords', 'id')
# 7400 keywords
# 2648 combination of keywords

In [29]:
# plot the frequency of appearances of production companies
keywords_frequency_dict = dict()
for collection, frequency in set_unique_combination_values.iteritems():
    for item in ast.literal_eval(collection):
        keyword = item['name']
        previous_frequency = keywords_frequency_dict.get(keyword, 0)
        keywords_frequency_dict[keyword] = previous_frequency + frequency
keywords_frequency_dict

{'independent film': 155,
 'woman director': 175,
 'duringcreditsstinger': 134,
 'suspense': 67,
 'biography': 77,
 'sport': 82,
 'dystopia': 73,
 'american football': 9,
 'musical': 49,
 'romantic comedy': 25,
 'bicycle': 4,
 'cycling': 2,
 'bicycle race': 2,
 'skab under tv': 3,
 '√òverste hylde': 3,
 'foran': 2,
 'stand-up comedy': 6,
 'kung fu': 12,
 'hitman': 26,
 'found footage': 19,
 'animation': 17,
 'new york': 47,
 'monster': 26,
 'skyscraper': 2,
 'mutant': 14,
 'restaurant': 15,
 'human animal relationship': 7,
 'mutation': 6,
 'tv station': 4,
 'dying and death': 34,
 'water': 5,
 'research station': 2,
 'fur': 3,
 'bat': 3,
 'current': 1,
 'electric shock': 3,
 'cleverness': 1,
 'gremlin': 2,
 'cowardliness': 5,
 'freedom of speech': 2,
 'social activism': 1,
 'dissident': 1,
 'chinese communists': 3,
 'sichuan earthquake': 1,
 'contemporary art': 1,
 'twitter': 1,
 'social media': 1,
 'based on novel': 111,
 'captain': 3,
 'boat': 9,
 'obsession': 17,
 'shipwreck': 10,
 

In [30]:
famous_keywords =[name for name, frequency in keywords_frequency_dict.items() if frequency > 30]
print(f'For 3000 rows, nb of keywords which appear more than 30 times: {len(famous_keywords)}')

For 3000 rows, nb of keywords which appear more than 30 times: 52


#### Crew

In [31]:
dict_jobs_set = dict()
for index, movie_crew in train_df['crew'].iteritems():
    try:
        list_characters = ast.literal_eval(movie_crew)
        for character in list_characters:
            if character['job'] == 'Director':
                previous_set = dict_jobs_set.get('Director', set())
                previous_set.add(character['id'])
                dict_jobs_set['Director'] = previous_set
            elif character['job'] == 'Producer':
                previous_set = dict_jobs_set.get('Producer', set())
                previous_set.add(character['id'])
                dict_jobs_set['Producer'] = previous_set
    except ValueError:
        pass
print(f"Nb of unique directors for 3000 rows:{len(dict_jobs_set['Director'])}")
print(f"Nb of unique producers for 3000 rows:{len(dict_jobs_set['Producer'])}")

Nb of unique directors for 3000 rows:2043
Nb of unique producers for 3000 rows:3554
